In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
df = pd.read_csv('../../output/seahorse_eval.csv')
df2 = pd.read_csv('../../output/seahorse_ds_2.csv')

In [ ]:
df = df[~df['metadata/Decoding config'].isna()]

# df = df[~df['metadata/Decoding config'].str.contains('short')]
display(df)
print(df.columns)


# tasks names
tasks_names = [c.split('/')[1] for c in df.columns if "SH" in c.split('/')[0]]
print(tasks_names)

In [ ]:

COL = "H(text|summary)"
COL2 = "H(text)"
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=False, sharex=False)
# order by I(summary -> text)

# same for df
df = df.sort_values(by=COL, ascending=False)
sns.barplot(data=df, x="metadata/Model name", y=COL, ax=ax2)

# set ylim based on the max and min
ax2.set_ylim([df[COL].min() - 0.05, df[COL].max() + 0.05])
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)


df = df.sort_values(by=COL2, ascending=False)
sns.barplot(data=df, x="metadata/Model name", y=COL2, ax=ax1)

# set ylim based on the max and min
ax1.set_ylim([df[COL2].min() - 0.05, df[COL2].max() + 0.05])
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)


In [ ]:

# order by I(summary -> text)
COL = "H(text|summary)"

df2 = df2.sort_values(by=COL, ascending=False)
sns.barplot(data=df2, x="metadata/Model name", y=COL, ax=ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)


# same for df
df = df.sort_values(by=COL, ascending=False)
sns.barplot(data=df, x="metadata/Model name", y=COL, ax=ax2)
# lim 55 65
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)

fig


In [ ]:


df['source'] = df["Unnamed: 0"]





In [ ]:

def custom_reg_plot(data, x=None, y=None, hue=None, **kwargs):
    ax = plt.gca()
    sns.regplot(data=data, x=x, y=y, ci=95, scatter=False, ax=ax, x_ci='sd')
    sns.scatterplot(data=data, x=x, y=y, hue=hue, alpha=1, s=100, ax=ax, **kwargs, palette='mako')
    return ax
    
    


fig, axes = plt.subplots(2, 3, figsize=(20, 10), sharey=False, sharex=False, dpi=300)
# COL = "H(summary|text)"

axes = axes.flatten()
df['N-MI_1'] = df['I(text -> summary)'] / df['H(summary)']
df['I(summary -> text) / H(text)'] = df['I(summary -> text)'] / df['H(text)']
# info diff:
df["I(text -> summary) - I(summary -> text)"] = df['I(text -> summary)'] - df['I(summary -> text)']
df["I(summary -> text) - I(text -> summary)"] = df['I(summary -> text)'] - df['I(text -> summary)']
COL = "I(summary -> text) / H(text)"
for k, task in enumerate(tasks_names):
        print(task)
        sns.set_theme(style="whitegrid")
        
        # sns.regplot(data=df, x="I(summary -> text)", y=f"SH/{task}", ci=95, scatter=False, ax=axes[tasks_names.index(task)], x_ci='sd')*
        # with correlation coefficient
        sns.regplot(data=df, x=COL, y=f"SH/{task}", ci=95, scatter=False, ax=axes[tasks_names.index(task)], x_ci='sd')
        
        # annotate with the correlation coefficient:
        corr = df[COL].corr(df[f"SH/{task}"])
        axes[tasks_names.index(task)].annotate(f"r={corr:.2f}", xy=(0.85, 0.05), xycoords='axes fraction', fontsize=16)
        
        
        sns.scatterplot(data=df, x=COL, y=f"SH/{task}", style="metadata/Model name", hue="metadata/Model name", alpha=1, s=300, ax=axes[tasks_names.index(task)], palette='mako')
        axes[tasks_names.index(task)].set_title(task, fontsize=20)
        
        # renames x and y labels
        axes[tasks_names.index(task)].set_xlabel(COL, fontsize=16)
        axes[tasks_names.index(task)].set_ylabel("Human Judgement", fontsize=16)
        
        # legend fontsize
        axes[tasks_names.index(task)].legend(fontsize=14)
        
    # remove legends
        if k != 1:
            axes[k].get_legend().remove()
            # put legend on bottom right
            
# put a single legend below the center of the whole figure

fig.legend(*axes[1].get_legend_handles_labels(), loc='lower center', ncol=3, fontsize=16, bbox_to_anchor=(0.5, -0.12))
axes[1].get_legend().remove()

fig.tight_layout()
path = f"../../../papers/Mutual-information-for-summarization/img/seahorse_ds/"
Path(path).mkdir(parents=True, exist_ok=True)

fig.savefig("../../../papers/Mutual-information-for-summarization/img/seahorse_ds/correlation_with_human_judgement.png", dpi=300, bbox_inches='tight')


        



In [ ]:
    df



In [ ]:

# List of datasets
datasets = df['metadata/Dataset name'].dropna().unique()

for dataset in datasets:
    
    # plot I(summary -> text) for each model
    sns.set_theme(style="whitegrid")
    # keep only top_p_sampling
    # df_top_p = df[df['metadata/Decoding config'] == 'top_p_sampling']
    # select dataset
    df_top_p = df[df['metadata/Dataset name'] == dataset]
    df_top_p = df[df['metadata/Decoding size'] == 50]
    
    # sort by I(summary -> text)
    df_top_p = df_top_p.sort_values(by="I(summary -> text)", ascending=False)
    
    ax =sns.barplot(data=df_top_p, y="metadata/Model name", x="I(summary -> text)", orient="h")
    
    # change y title to be more readable
    ax.set(ylabel="Model name")
    
    # change xlim based to be a little bit less than the min and a little bit more than the max
    ax.set_xlim([df_top_p['I(summary -> text)'].min() - 5, df_top_p['I(summary -> text)'].max() + 5])
    
    # save figure
    #plt.savefig(f"../../../papers/Mutual-information-for-summarization/img/model_comparison/{dataset}_top_p.png", dpi=300, bbox_inches='tight')
    # plt.clf()
    


In [ ]:




ddf = df.drop('Unnamed: 0', axis=1)
# df = df.set_index([metadata for metadata in df.columns if "metadata" in metadata])
ddf = ddf.melt(id_vars=[metadata for metadata in ddf.columns if "metadata" in metadata] ,var_name="Score", value_name="Value")







In [ ]:

def custom_reg_plot(data, x=None, y=None, hue=None, **kwargs):
    ax = plt.gca()
    sns.regplot(data=data, x=x, y=y, ci=95, scatter=False, ax=ax, x_ci='sd')
    sns.scatterplot(data=data, x=x, y=y, hue=hue, alpha=1, s=100, ax=ax, **kwargs, palette='mako')
    return ax




fig, axes = plt.subplots(2, 3, figsize=(20, 10), sharey=False, sharex=False, dpi=300)
# COL = "H(summary|text)"

axes = axes.flatten()
df['N-MI_1'] = df['I(text -> summary)'] / df['H(summary)']
df['I(summary -> text) / H(text)'] = df['I(summary -> text)'] / df['H(text)']
# info diff:
df["I(text -> summary) - I(summary -> text)"] = df['I(text -> summary)'] - df['I(summary -> text)']
df["I(summary -> text) - I(text -> summary)"] = df['I(summary -> text)'] - df['I(text -> summary)']
COL =  "H(text)"
for k, task in enumerate(tasks_names):
    print(task)
    sns.set_theme(style="whitegrid")

    # sns.regplot(data=df, x="I(summary -> text)", y=f"SH/{task}", ci=95, scatter=False, ax=axes[tasks_names.index(task)], x_ci='sd')*
    # with correlation coefficient
    sns.regplot(data=df, x=COL, y=f"SH/{task}", ci=95, scatter=False, ax=axes[tasks_names.index(task)], x_ci='sd')

    # annotate with the correlation coefficient:
    corr = df[COL].corr(df[f"SH/{task}"])
    axes[tasks_names.index(task)].annotate(f"r={corr:.2f}", xy=(0.85, 0.05), xycoords='axes fraction', fontsize=16)


    sns.scatterplot(data=df, x=COL, y=f"SH/{task}", style="metadata/Model name", hue="metadata/Model name", alpha=1, s=300, ax=axes[tasks_names.index(task)], palette='mako')
    axes[tasks_names.index(task)].set_title(task, fontsize=22)

    # renames x and y labels
    axes[tasks_names.index(task)].set_xlabel(COL, fontsize=20)
    axes[tasks_names.index(task)].set_ylabel("Human Judgement", fontsize=20)

    # legend fontsize
    axes[tasks_names.index(task)].legend(fontsize=14)

    # remove legends
    if k != 1:
        axes[k].get_legend().remove()
        # put legend on bottom right

# put a single legend below the center of the whole figure

fig.legend(*axes[1].get_legend_handles_labels(), loc='lower center', ncol=3, fontsize=18, bbox_to_anchor=(0.5, -0.12))
axes[1].get_legend().remove()

fig.tight_layout()
path = f"../../../papers/Mutual-information-for-summarization/img/seahorse_ds/"
Path(path).mkdir(parents=True, exist_ok=True)

fig.savefig("../../../papers/Mutual-information-for-summarization/img/seahorse_ds/correlation_with_human_judgement_htext.png", dpi=300, bbox_inches='tight')

